In [1]:
import numpy as np
import pandas as  pd  
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import StandardScaler

C:\Users\sakth\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Preprocessing: Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert target labels to one-hot encoded format
num_classes = len(np.unique(y))
y_train_onehot = np.eye(num_classes)[y_train]

# Define the neural network architecture
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        # Forward propagation
        self.hidden_output = self.sigmoid(np.dot(X, self.weights_input_hidden) + self.bias_hidden)
        self.output = self.sigmoid(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)
        return self.output
    
    def backward(self, X, y, output, learning_rate):
        # Backpropagation
        error = y - output
        output_delta = error * self.sigmoid_derivative(output)
        
        error_hidden = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = error_hidden * self.sigmoid_derivative(self.hidden_output)
        
        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * learning_rate
    
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss}")
    
    def predict(self, X):
        # Make predictions
        hidden_output = self.sigmoid(np.dot(X, self.weights_input_hidden) + self.bias_hidden)
        output = self.sigmoid(np.dot(hidden_output, self.weights_hidden_output) + self.bias_output)
        return np.argmax(output, axis=1)

# Define hyperparameters
input_size = X_train.shape[1]
hidden_size = 5
output_size = num_classes  # Adjusted to match the number of classes in the dataset
epochs = 1000
learning_rate = 0.01

# Initialize and train the neural network
model = NeuralNetwork(input_size, hidden_size, output_size)
model.train(X_train, y_train_onehot, epochs, learning_rate)

# Make predictions
y_pred = model.predict(X_test)
print("Predictions:", y_pred)


Epoch 0, Loss: 0.3350532546763256
Epoch 100, Loss: 0.022994530240604077
Epoch 200, Loss: 0.016305541970661974
Epoch 300, Loss: 0.01325346861404126
Epoch 400, Loss: 0.011457518276625616
Epoch 500, Loss: 0.010170157994652032
Epoch 600, Loss: 0.008840360463701482
Epoch 700, Loss: 0.007779193790422237
Epoch 800, Loss: 0.006951470143481528
Epoch 900, Loss: 0.006303896949852027
Predictions: [1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 1 0]


In [3]:
print(accuracy_score(y_test,y_pred))

0.9912280701754386


In [4]:
print(confusion_matrix(y_test,y_pred))

[[42  1]
 [ 0 71]]
